In [1]:
# 기본 라이브러리 임포트
import pandas as pd             
import numpy as np             
import matplotlib.pyplot as plt 
import seaborn as sns           
import gc                       # 가비지 컬렉션(메모리 해제)
import re                       # 정규 표현식 처리
from collections import defaultdict  # 기본값이 있는 딕셔너리 생성

# 경고 메시지 억제
import warnings
warnings.filterwarnings('ignore')   

# 그래프 스타일 설정
sns.set()                           # seaborn 기본 스타일 적용

# matplotlib 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 한글 폰트 설정
# plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['figure.figsize'] = (12, 6)       # 그림 크기 설정 (가로, 세로)
plt.rcParams['font.size'] = 14                 # 폰트 크기 설정
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지

# 결측치 시각화 라이브러리 임포트
import missingno                          # 결측치 분포를 시각화하는 유틸리티

# 범주형 변수 레이블 인코딩을 위한 도구 임포트
from sklearn.preprocessing import LabelEncoder

# 중복 조합 생성에 사용할 product 함수 임포트
from itertools import product

# 회귀 및 통계 분석을 위한 statsmodels 임포트
import statsmodels.api as sm

# 다중공선성 진단용 VIF 계산 함수 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv('병합_selected_features.csv')
df

,ID,Segment,이용금액_오프라인_R3M,이용금액대_ord,정상청구원금_B2M,이용금액_R3M_신용체크,연속유실적개월수_기본_24M_카드,미이용_CA,정상청구원금_B0M,최대이용금액_체크_R12M,...,RP후경과월_전기,이용금액_카드론_R12M,이용건수_체크_R6M,RV_평균잔액_R12M,이용개월수_할부_유이자_R6M,_3순위쇼핑업종_이용금액,잔액_할부_유이자_B0M,잔액_현금서비스_B2M,이용가능여부_해외겸용_본인,RV_최대잔액_R12M
0,TRAIN_000000,D,11756,6,15251,-454,17,0,15067,998,...,6,0,0,0,0,0,0,30640,0,0
1,TRAIN_000001,E,12128,5,2776,7089,17,0,2222,0,...,6,0,0,0,0,517,1491,0,0,0
2,TRAIN_000002,C,24370,6,23325,27336,8,0,26184,0,...,6,0,0,1255,0,1172,0,27654,1,4587
3,TRAIN_000003,D,12529,6,18808,4270,24,0,20959,0,...,6,0,0,0,4,682,2092,28938,1,0
4,TRAIN_000004,E,0,2,0,9385,0,1,639,3910,...,6,0,55,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,TRAIN_399995,E,0,1,0,10755,0,1,19,5072,...,6,0,76,0,0,0,0,0,1,0
399996,TRAIN_399996,D,25205,6,14844,27636,17,1,13462,0,...,6,0,0,0,0,346,0,0,1,0
399997,TRAIN_399997,C,13267,3,6862,23187,24,1,7049,0,...,6,0,0,0,0,305,0,0,1,0
399998,TRAIN_399998,E,0,1,0,0,0,1,0,0,...,6,0,0,0,0,0,0,0,0,0


In [3]:
# ID와 Segment를 먼저 제거하고, 숫자형 컬럼만 선택합니다
df_vif = (
    df
    .drop(columns=['ID', 'Segment'])
    .select_dtypes(include=[np.number])
    .copy()
)

threshold = 10
removed_features = []

In [ ]:
while True:
    # 상수항을 추가하고 결측값을 0으로 채웁니다
    X = sm.add_constant(df_vif).fillna(0)
    # 각 변수별 VIF를 계산합니다
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif_df = pd.DataFrame({'feature': X.columns, 'vif': vif})
    # 상수항은 제외합니다
    vif_df = vif_df[vif_df['feature'] != 'const']
    # 가장 높은 VIF와 해당 변수를 찾습니다
    max_vif = vif_df['vif'].max()
    max_feature = vif_df.loc[vif_df['vif'].idxmax(), 'feature']
    # 기준을 넘는 변수가 있으면 제거하고 리스트에 추가합니다
    if max_vif > threshold:
        removed_features.append(max_feature)
        df_vif.drop(columns=[max_feature], inplace=True)
    else:
        break

In [ ]:
# 제거된 변수 목록과 최종 VIF 결과를 출력합니다
print("제거된 변수들:", removed_features)
print(vif_df.reset_index(drop=True))